In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
import pickle as pkl
import matplotlib

In [ ]:
# load mind articled_mind.pkl
with open("../data/mind/articles_mind.pickle", "rb") as f:
    articles = pkl.load(f)

In [ ]:
with open("../results/mind_results_k@10.pkl", "rb") as f:
    results = pkl.load(f)

print(len(results['topic_calibration']['naml']))
results_mean = results.map(
    lambda x: np.mean(x) if isinstance(x, (list, np.ndarray)) and len(x) > 0 else np.nan
)

display(results_mean)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

legend_name_map = {
    'pop': 'Popularity',
    'random': 'Random',
    'npa': 'NPA',
    'nrms': 'NRMS',
    'lstur': 'LSTUR',
    'naml': 'NAML',
}
ordered_recommenders = ['lstur', 'naml', 'nrms', 'npa', 'pop', 'random']
metric_specs = [
    {'label': 'Topic Calibration', 'x_label': 'Topic Calibration', 'path': Path('../results/mind_topic_tradeoff_k@10.csv')},
    {'label': 'Subtopic Calibration', 'x_label': 'Subtopic Calibration', 'path': Path('../results/mind_subtopic_tradeoff_k@10.csv')},
    {'label': 'Complexity Calibration', 'x_label': 'Complexity Calibration', 'path': Path('../results/mind_complexity_tradeoff_k@10.csv')},
    {'label': 'Fragmentation', 'x_label': 'Fragmentation', 'path': Path('../results/mind_fragmentation_tradeoff_k@10.csv')},
    {'label': 'Activation', 'x_label': 'Activation', 'path': Path('../results/mind_activation_tradeoff_k@10.csv')},
]

palette = matplotlib.colormaps['tab10']
color_map = {rec: palette(i) for i, rec in enumerate(ordered_recommenders)}

fig, axes = plt.subplots(2, 3, figsize=(18, 12), sharey=True)
axes = axes.flatten()
legend_handles = {}

for ax, spec in zip(axes, metric_specs):
    tradeoff_df = pd.read_csv(spec['path'])
    tradeoff_df = tradeoff_df.dropna(subset=['ndcg', 'divergence'])
    tradeoff_df = tradeoff_df.sort_values(['recommender', 'lambda'])
    plot_df = tradeoff_df[tradeoff_df['recommender'] != 'incorrect_random']

    for rec in ordered_recommenders:
        group = plot_df[plot_df['recommender'] == rec]
        if group.empty:
            continue
        display_name = legend_name_map.get(rec, rec)

        # Add a tiny jitter when multiple lambdas land on the exact same point (shift right only)
        jittered = group.copy()
        seen_coords = {}
        jitter_step = 2.5e-3
        jitter_margin = 1e-2
        for idx, row in jittered.iterrows():
            key = (
                round(row['divergence'] / jitter_margin),
                round(row['ndcg'] / jitter_margin),
            )
            offset = seen_coords.get(key, 0)
            if offset:
                jittered.at[idx, 'divergence'] = row['divergence'] + jitter_step * offset
            seen_coords[key] = offset + 1

        line, = ax.plot(
            jittered['divergence'],
            jittered['ndcg'],
            linewidth=3.5,
            marker='o',
            markersize=8,
            color=color_map[rec],
            label=display_name,
        )
        legend_handles[display_name] = line

    ax.set_xlabel(spec['x_label'], fontsize=18)
    ax.set_ylabel('NDCG@10', fontsize=18)
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.grid(True, linestyle='--', alpha=0.5)

for ax in axes[len(metric_specs):]:
    ax.axis('off')

fig.legend(
    legend_handles.values(),
    legend_handles.keys(),
    loc='lower center',
    ncol=3,
    fontsize=16,
    title='Recommender',
    title_fontsize=16,
    bbox_to_anchor=(0.5, -0.04),
)
plt.tight_layout(rect=[0, 0.08, 1, 1])
plt.subplots_adjust(bottom=0.12)
plt.show()